In [8]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from PIL import Image
from tqdm import tqdm as tqdm
import pandas as pd
import os

In [9]:


class CustomResNet50Regression(nn.Module):
    def __init__(self, output_size=1, fine_tune=True):
        super(CustomResNet50Regression, self).__init__()
        # Cargar la ResNet50 preentrenada
        self.resnet50 = models.resnet50(pretrained=True)
        # Reemplazar la última capa para regresión
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, output_size)
        # Congelar capas convolucionales si no se desea fine-tuning
        if not fine_tune:
            for param in self.resnet50.parameters():
                param.requires_grad = False
            # Solo las capas de la fc estarán entrenables
            for param in self.resnet50.fc.parameters():
                param.requires_grad = True

    def forward(self, x):
        return self.resnet50(x)

    def train_model(self, train_loader, val_loader, epochs, learning_rate, fine_tune=True, device='cuda'):
        self.to(device)
        criterion = nn.MSELoss()  # Función de pérdida para regresión
        optimizer = optim.Adam(self.parameters() if fine_tune else self.resnet50.fc.parameters(), lr=learning_rate)

        for epoch in range(epochs):
            # Entrenamiento
            self.train()
            train_loss = 0.0
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            # Validación
            self.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = self(inputs)
                    loss = criterion(outputs, targets)
                    val_loss += loss.item()

            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")




In [10]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar la imagen
    transforms.Grayscale(num_output_channels=3),  # Convertir imágenes en escala de grises a 3 canales (RGB)
    transforms.ToTensor(),  # Convertir la imagen a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalización
])
y_train = torch.tensor(pd.read_csv('../data/preprocessed/train/Y_train.csv')['Day30'].values, dtype=torch.float32)
y_test = torch.tensor(pd.read_csv('../data/preprocessed/test/Y_test.csv')['Day30'].values, dtype=torch.float32)

train_images = []
test_images = []

train_img_dir = '../data/preprocessed/train/train_imgs'
train_img_names = os.listdir(train_img_dir)

for img_name in tqdm(train_img_names):
    img_path = os.path.join(train_img_dir, img_name)
    image = Image.open(img_path)
    image = data_transforms(image)  # Aplicar las transformaciones
    train_images.append(image)
train_images_tensor = torch.stack(train_images)

test_img_dir = '../data/preprocessed/test/test_imgs'
test_img_names = os.listdir(test_img_dir)

for img_name in tqdm(test_img_names):
    img_path = os.path.join(test_img_dir, img_name)
    image = Image.open(img_path)
    image = data_transforms(image)  # Aplicar las transformaciones
    test_images.append(image)
test_images_tensor = torch.stack(test_images)


100%|██████████| 2034/2034 [00:28<00:00, 72.54it/s]


In [11]:
model = CustomResNet50Regression(output_size=1, fine_tune=True)
train_dataset = torch.utils.data.TensorDataset(train_images_tensor, y_train)
test_dataset = torch.utils.data.TensorDataset(test_images_tensor, y_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)
 
model.train_model(train_loader, test_loader, epochs=10, learning_rate=0.001, fine_tune=False, device='cpu')

/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


: 